Известно:

K = 8


V = 5000

N = 2400

lambda(day) = 8

lambda(night) = 0.5

B = 7

Первый пункт:

In [5]:
import numpy as np
import random



# Количество концентраторов
k = 8
# Интесивность входного потока
lambda_day = 8
# Интенсивность ночного потока
lambda_night = 0.5
# Максимальный размер очереди
m = 0
# Интенсивность выходного потока
mu = 2.08
# Количество обслуженных заявок в единицу времени
p_day = lambda_day/mu
p_night = lambda_night/mu
# Предельная вероятность
p0 = 0
f = 1
for i in range(1, k+1):
    f *= i
    p0 += (p_day**i)/f
p0 = (p0 + 1 + ((p_day**(k+1)/k*f) * (1-(p_day/k)**m)/(1-p_day/k)))**(-1)
# Вероятность отказа
pd = (p_day**(k+m)/((k**m)*f))*p0

p0 = 0
f = 1
for i in range(1, k+1):
    f *= i
    p0 += (p_night**i)/f
p0 = (p0 + 1 + (((p_night**(k+1))/(k*f)) * (1-(p_night/k)**m)/(1-p_night/k)))**(-1)
# Вероятность отказа
pe = (p_night**(k+m)/((k**m)*f))*p0

print('Вероятность отказа днём:', pd)
print('Вероятность отказа ночью:', pe)

Вероятность отказа днём: 0.025812595940994068
Вероятность отказа ночью: 2.1743783718213567e-10


Второй пункт:

In [9]:
import numpy as np
import random

def alg(lambda_day, n_packets):
    # Инициализация генеротора случайных чисел
    np.random.seed(100)
    # Вероятность блокировки
    p_refuse = 0
    # Количество повторов в методе Монте-Карло
    n_rep = 100
    # Имитационное моделирование
    for i in range(n_rep):
    # Количество концентраторов
        k = 8
    # Текущее время
        t = 0
    # текущий размер очереди
        queue = 0
    # Максимальный размер очереди
        m = 0
    # Интенсивность выходного потока
        mu = 2.08
    # время, когда прибор освободится
        t_free = []
        for j in range(k):
            t_free.append(0)
    # Количество потярянных пактеов
        n_lost = 0
        for f in range(n_packets):
    # время поступления нового пакета
            t += np.random.exponential(scale=1/lambda_day)
    # проверяем, свободен ли концентратор
            n = 0
            if queue > 0:
                for j in range(k):
                    while t_free[j] < t and queue > 0:
                        t_free[j] += np.random.exponential(scale=1/mu)
                        queue -= 1
            for h in range(k):
                if t_free[h] < t:
                    t_free[h] = t + np.random.exponential(scale=1/mu)
                    break
                else:
                    n += 1
            if n == k:
                if queue < m:
                    queue += 1
                else:
                    n_lost += 1
        p_refuse += n_lost/n_packets
    return p_refuse
# Оценка вероятности блокировки
# Количество повторов в методе Монте-Карло
n_rep = 100
p_refuse1 = alg(8, 460800) / n_rep
p_refuse2 = alg(0.5, 14400) / n_rep
print('Вероятность отказа днём: '+str(p_refuse1))
print('Вероятность отказа ночью: '+str(p_refuse2))

Вероятность отказа днём: 0.025785698784722227
Вероятность отказа ночью: 0.0


Я специально вбил соответствующие числа пакетов и днём и ночью и получил соответствующие результаты. Вероятность потери файла днём по методу Монте Карло равна вероятности потери файла днём в первом пункте. А вот тк вероятность потери ночью стремится к 0, то метод Монте Карло определил её как 0.

Третий пункт:

Тут я подставлял различные значения очереди и пришёл к тому, что оптимальный размер буфера будет 9 т.к. если размер
буфера будет 8, то P(отк) = 0.00011, что больше 0.0001

In [7]:
import numpy as np
import random


# Инициализация генеротора случайных чисел
np.random.seed(100)
# Количество повторов в методе Монте-Карло
n_rep = 1000
# Вероятность блокировки
p_refuse = 0
# Имитационное моделирование
for i in range(n_rep):
# Количество концентраторов
    k = 8
# Количество пакетов
    n_packets = 1000
# Текущее время
    t = 0
# текущий размер очереди
    queue = 0
# Интесивность входного потока
    lambda_day = 8
# Максимальный размер очереди
    m = 8
# Интенсивность выходного потока
    mu = 2.08
# время, когда прибор освободится
    t_free = []
    for j in range(k):
        t_free.append(0)
# Количество потярянных пактеов
    n_lost = 0
    for f in range(n_packets):
# время поступления нового пакета
        t += np.random.exponential(scale=1/lambda_day)
# проверяем, свободен ли концентратор
        n = 0
        if queue > 0:
            for j in range(k):
                while t_free[j] < t and queue > 0:
                    t_free[j] += np.random.exponential(scale=1/mu)
                    queue -= 1
        for h in range(k):
            if t_free[h] < t:
                t_free[h] = t + np.random.exponential(scale=1/mu)
                break
            else:
                n += 1
        if n == k:
            if queue < m:
                queue += 1
            else:
                n_lost += 1
    p_refuse += n_lost/n_packets

# Оценка вероятности блокировки
p_refuse /= n_rep
print('Вероятность отказа: '+str(p_refuse))

Вероятность отказа: 6.300000000000001e-05
